In [35]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [2]:
df = pd.read_excel("data/SpareItemDump.xlsx",engine="openpyxl")

In [93]:
df.dropna(subset = ["PurchaseOrderId"], inplace=True)
df.reset_index(drop=True, inplace=True)

In [94]:
df.dropna(subset = ["Make"], inplace=True)
df.reset_index(drop=True, inplace=True)

In [95]:
df["PurchaseOrderId"]=df["PurchaseOrderId"].astype(int)

In [96]:
df=df.assign(product=None)
for i in range(len(df)):
    df["product"][i] = str(df["Make"][i])+"_"+str(df["Model"][i])+"_"+str(df["ItemName"][i])

/home/narendra/anaconda3/envs/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [97]:
df

,RequestDate,RequestId,PurchaseOrderId,Vendor,Garage,Make,Model,SubCategory,Category,RequestedBrand,FulfilledBrand,Variant,ItemName,ItemQty,ItemUnitPrice,PartNbr,product
0,2023-07-01,303887,15791,Essarr Automotives - Bannerghatta,Canaraa Car Care Centre,Maruti Suzuki,Omni,NaN,NaN,oem,mgp,Std,Plug Wire,1.0,743.60,33700M79020,Maruti Suzuki_Omni_Plug Wire
1,2023-07-01,303890,15790,Autoverse Mobility Pvt Ltd,Express Car Care,Hyundai,Eon,NaN,NaN,any,Purolator,Nill,Air Filter,5.0,164.56,PI-2272,Hyundai_Eon_Air Filter
2,2023-07-01,303893,15936,Suma Enterprises,Arodhana Synergy Private Limited,Mahindra,XUV500,NaN,NaN,oem,Mahindra,R FWD,"Bracket, Radiator Support Lower",1.0,12438.00,0102AW0400NN,"Mahindra_XUV500_Bracket, Radiator Support Lower"
3,2023-07-01,303895,15794,Essarr Automotives - Bannerghatta,Laksh,Maruti Suzuki,Swift,NaN,NaN,oem,MGP,V,Alternator Belt,2.0,382.80,95141M86JB0,Maruti Suzuki_Swift_Alternator Belt
4,2023-07-01,303896,15793,Essarr Automotives - Bannerghatta,Laksh,Maruti Suzuki,Swift,NaN,NaN,oem,MARUTI,V,Air Filter,5.0,171.60,13780M76M00,Maruti Suzuki_Swift_Air Filter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5186,2023-10-26,311379,18233,Grand Auto Spares,Motorlyfe Automotive,Hyundai,i20,Spark Plug,Electrical Components,oes,NGK,1.2L (83 h.p.),Spark Plug Set Of 4,1.0,180.34,LKR7E,Hyundai_i20_Spark Plug Set Of 4
5187,2023-10-26,311387,18239,Essarr Automotives - BG Road,V R Automotives,Maruti Suzuki,Alto 800,Door Handle,Interior and Comfort,oem,Maruti Suzuki,0.8L LX (TYPE 2) (47 h.p.),Rear Outer Door Handle - LH,1.0,237.60,82802M68K50-5PK,Maruti Suzuki_Alto 800_Rear Outer Door Handle ...
5188,2023-10-26,311387,18239,Essarr Automotives - BG Road,V R Automotives,Maruti Suzuki,Alto 800,Door Handle,Interior and Comfort,oem,Maruti Suzuki,0.8L LX (TYPE 2) (47 h.p.),Outer Door Handle - RH,1.0,237.60,82801M68K50-5PK,Maruti Suzuki_Alto 800_Outer Door Handle - RH
5189,2023-10-26,311388,18238,Essarr Automotives - BG Road,Wrap Auto Shop And Service Center,Maruti Suzuki,Swift,Outside Mirror Cover,Body,oem,Maruti Suzuki,1.3L LDI (TYPE 1) (74 h.p.),Mirror Cap - RH,1.0,206.80,84728M74LA0-5PK,Maruti Suzuki_Swift_Mirror Cap - RH


In [98]:
d={}
for i in range(len(df)):
    if df["PurchaseOrderId"][i] in d:
        (d[df["PurchaseOrderId"][i]])[df["product"][i]] =1
            
    else:
        d[df["PurchaseOrderId"][i]]={df["product"][i]:1}

In [99]:
d

{15791: {'Maruti Suzuki_Omni_Plug Wire': 1},
 15790: {'Hyundai_Eon_Air Filter': 1},
 15936: {'Mahindra_XUV500_Bracket, Radiator Support Lower': 1},
 15794: {'Maruti Suzuki_Swift_Alternator Belt': 1},
 15793: {'Maruti Suzuki_Swift_Air Filter': 1},
 15797: {'Hyundai_i10_Air Filter': 1},
 15796: {'Hyundai_i10_Air Filter': 1},
 15795: {'Maruti Suzuki_Swift_Air Filter': 1},
 15808: {'Maruti Suzuki_Swift_Fender Liner - L/H Front': 1,
  'Maruti Suzuki_Swift_Fender Liner - R/H Front': 1},
 15800: {'Audi_Q5_Belt': 1},
 15806: {'Maruti Suzuki_Omni_Wind Shield - Front': 1},
 15805: {'Maruti Suzuki_Swift_Front Brake pad': 1},
 15802: {'Chevrolet_Tavera Neo 3_Cam Shaft Oil Seal': 1},
 15803: {'Tata_Indica_Front Brake pad': 1},
 15804: {'Toyota_Etios_Front Brake pad': 1},
 15798: {'Maruti Suzuki_Eeco_Engine Oil - Taxi Special - 5W30 (4L)': 1,
  'Maruti Suzuki_Eeco_Coolant (Green)': 1,
  'Maruti Suzuki_Eeco_Brake Oil': 1},
 15799: {'Maruti Suzuki_Eeco_Oil Filter': 1,
  'Maruti Suzuki_Eeco_Air Filter'

In [100]:
li=[]
for i in d:
    for j in d[i]:
        li.append([i,j,d[i][j]])
data = pd.DataFrame(li,columns=["PurchaseOrderId","product","purchase_count"])

In [101]:
data

,PurchaseOrderId,product,purchase_count
0,15791,Maruti Suzuki_Omni_Plug Wire,1
1,15790,Hyundai_Eon_Air Filter,1
2,15936,"Mahindra_XUV500_Bracket, Radiator Support Lower",1
3,15794,Maruti Suzuki_Swift_Alternator Belt,1
4,15793,Maruti Suzuki_Swift_Air Filter,1
...,...,...,...
5186,18233,Hyundai_i20_Spark Plug Set Of 4,1
5187,18239,Maruti Suzuki_Alto 800_Rear Outer Door Handle ...,1
5188,18239,Maruti Suzuki_Alto 800_Outer Door Handle - RH,1
5189,18238,Maruti Suzuki_Swift_Mirror Cap - RH,1


In [102]:
counts = data["PurchaseOrderId"].value_counts()
data = data[data["PurchaseOrderId"].isin(counts[counts >= 3].index)]

In [103]:
data

,PurchaseOrderId,product,purchase_count
16,15798,Maruti Suzuki_Eeco_Engine Oil - Taxi Special -...,1
17,15798,Maruti Suzuki_Eeco_Coolant (Green),1
18,15798,Maruti Suzuki_Eeco_Brake Oil,1
19,15799,Maruti Suzuki_Eeco_Oil Filter,1
20,15799,Maruti Suzuki_Eeco_Air Filter,1
...,...,...,...
5176,18236,Mahindra_Jeeto_Axle Nut,1
5183,18233,Hyundai_i20_Front Brake pad,1
5184,18233,Hyundai_i20_Wheel Brake Cylinder,1
5185,18233,Hyundai_i20_AC Filter,1


In [104]:
data.to_csv("new_data.csv")

In [3]:
data = pd.read_csv("new_data.csv")

In [4]:
data

,PurchaseOrderId,product,purchase_count
0,15798,Maruti Suzuki_Eeco_Engine Oil - Taxi Special -...,1
1,15798,Maruti Suzuki_Eeco_Coolant (Green),1
2,15798,Maruti Suzuki_Eeco_Brake Oil,1
3,15799,Maruti Suzuki_Eeco_Oil Filter,1
4,15799,Maruti Suzuki_Eeco_Air Filter,1
...,...,...,...
2981,18236,Mahindra_Jeeto_Axle Nut,1
2982,18233,Hyundai_i20_Front Brake pad,1
2983,18233,Hyundai_i20_Wheel Brake Cylinder,1
2984,18233,Hyundai_i20_AC Filter,1


In [51]:
purchase_id= 1
make  = "Tata"
model = "Nano"
product_name = "Fan Assembly"

prod = str(make)+"_"+str(model)+"_"+str(product_name)

lol=pd.DataFrame(columns=["PurchaseOrderId","product","purchase_count"])

In [54]:
lol.loc[0] = [purchase_id,prod, 1]

In [55]:
lol

,PurchaseOrderId,product,purchase_count
0,1,Tata_Nano_Fan Assembly,1


In [56]:
data_pivot = data.pivot(index = 'product', columns = 'PurchaseOrderId', values = 'purchase_count').fillna(0)
data_matrix = csr_matrix(data_pivot.values)

In [57]:
data_matrix

<2212x610 sparse matrix of type '<class 'numpy.float64'>'
	with 2987 stored elements in Compressed Sparse Row format>

In [21]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(data_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [24]:
query_index = np.random.choice(data_pivot.shape[0])
print(query_index)

1995


In [34]:
data_pivot.iloc[query_index,:].values.reshape(1, -1)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [31]:
query_index=2211

In [32]:

distances, indices = model_knn.kneighbors(data_pivot.iloc[query_index,:].values.reshape(1, -1))

In [33]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for:- {0}:\n'.format(data_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, data_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for:- Volkswagen_Vento_Top Strut Mount Kit:

1: Volkswagen_Vento_Ball Joint Set, with distance of 0.0:
2: Volkswagen_Vento_Front Shock Absorber, with distance of 0.0:
3: Volkswagen_Vento_Lower Arm - RH, with distance of 0.0:
4: Volkswagen_Vento_Front Stabilizer Bush, with distance of 0.0:
